# Experiment Notebook 
#### This notebook is meant to experiment with different parameters, architectures, and normalization methods associated with the gRNA2knockdown Library.
#### The basic characteristic of the model as developed by Enoch Yeung are still preserved in the __main__ function of the module.

In [ ]:
import os
import numpy as np
import platereadertools as prt
import gRNA2knockdown as g2k 

import matplotlib.pyplot as plt 

#### First Let's import or data, the description of which is below:
- The sequencing data is imported as a csv and contains gRNA designs which tile the target (GFP) with 3 bp of overlap.
- The timeseries data is imported as a tsv text file generated BioTek's Gen5 software and from data collected on Synergy H1s.
- The experimental design is:
  - Total run time = 18 hours
  - Timestep = 3 min.
  - Reads:
    - OD600: Absorbance at 600nm
    - Flourescence: Exitation = , Emmission = .
  - Runs where conducted with breath-easy film to reduce moisture loss.